# Home Assignment 1
Due by 8th May, 2024 at 23:59 CEST

## Neural Network Basics

We want to create a 2 layer NN, which means we want to calculate  $y = W_2 * ReLU(W_1 * x + b_1) + b_2$

Complete the TODOs below to create such a NN.

Since you will be needing to compute the gradients w.r.t. all parameters, you may look into online resources for help. Please cite or link any online recources you do use.

You are allowed to change any existing parts, however the code has to remain easy to understand and well documented.

In [67]:
import numpy as np
import matplotlib.pyplot as plt

def relu(x):
    """
    ReLU activation function
    
    Parameters:
        x (np.ndarray): numpy array with shape (m, n) where m is the number of dimensions and n is the number of points
        
    Returns:
        x' (np.ndarray): return value of the pointwise ReLU application
    """
    return np.maximum(0, x)


def relu_grad(x):
    # TODO: Write a function given a numpy array that calculates the gradient of the ReLU function w.r.t. `x`
    # TODO: Also write the derivation of the gradient in the PDF file In the implementation you may simply use the final derivation.
    # Hint: The function should return a numpy array of the same dimension that `x` has, but only containing 0 or 1
    arr = np.zeros(x.shape)
    return np.greater(x, arr).astype(int)


In [68]:
class NumPyNeuralNet:
    
    def __init__(self, dim_in, dim_hidden, dim_out):
        self.dim_in = dim_in
        self.dim_hidden = dim_hidden
        self.dim_out = dim_out
        
        # TODO: Randomly initialize the weight matrices W_1, W_2 and biases b_1, b_2
        # Hint: use np.random.randn() and make sure to correctly set the dimensions 

        # Scale random sample with 0.01 according to lecture
        self.W_1 = 0.01 * np.random.randn(self.dim_in, self.dim_hidden)
        self.b_1 = 0.01 * np.random.randn(self.dim_hidden)
        self.W_2 = 0.01 * np.random.randn(self.dim_hidden, self.dim_out)
        self.b_2 = 0.01 * np.random.randn(self.dim_out)

    def predict(self, x):
        """
        Calculates the output of the neural network for the given x.
        
        Parameters:
            x (np.ndarray): input value numpy array
        
        Returns:
            y (np.ndarray): predicted output for `x`
        """
        # TODO: Calculate output self.out
        # Safe intermediate results as cache for later backpropagation
        self.h_1 = np.dot(x, self.W_1) + self.b_1
        self.h_1_act = relu(self.h_1)
        self.out = np.dot(self.h_1_act, self.W_2) + self.b_2
        return self.out
    
    
    def loss(self, x, y):
        """
        Calculates the Mean-Squared Error and returns the gradients w.r.t. to the parameters.
        
        Parameters:
            x (np.ndarray): input value numpy array with shape (self.dim_in, n)
            y (np.ndarray): ground truth value numpy array with shape (self.dim_out, n)
            
        Returns:
            loss (float): Mean-Squared-Error between predicted value on input points and ground truth value
            W_1_grad (np.ndarray): gradient w.r.t W_1   
            W_2_grad (np.ndarray): gradient w.r.t W_2  
            b_1_grad (np.ndarray): gradient w.r.t b_1   
            b_2_grad (np.ndarray): gradient w.r.t b_2   
        """
        # TODO: Calculate the loss (Mean-Squared-Error)
        # Hint: use np.square() and np.mean()

        y_pred = self.predict(x)
        loss = np.mean(np.square(y_pred - y))
        
        # TODO: Calculate all gradients w.r.t to the parameters
        # Hint: You need to calculate the gradients for each of the parameters by hand
        # TODO: Also write the derivation of the gradient in the PDF file. In the implementation you may simply use the final derivation.

        loss_derived = 2 * (self.out - y) / len(y)
        
        h_1_grad = np.dot(loss_derived, self.W_2.T) * relu_grad(self.h_1)
        
        W_2_grad = np.dot(self.h_1_act.T, loss_derived)
        b_2_grad = np.sum(loss_derived, axis=0)
        W_1_grad = np.dot(x.T, h_1_grad)
        b_1_grad = np.sum(h_1_grad, axis=0)

        return loss, W_1_grad, W_2_grad, b_1_grad, b_2_grad
         
    def train(self, x, y, lr=0.001, epochs=1000):
        """
        Train the neural network with gradient descent.
        
        Parameters:
            x (np.ndarray): input values
            y (np.ndarray): ground truth values
            lr (float): learning rate, default: 0.001
            epochs (int): number of epochs to train, default: 1000
            
        Returns:
            loss (float): Return the loss achieved after all epochs
        """
        # TODO: Keep track of the loss
        loss_history = []
        n = len(x)
        
        for epoch in range(epochs):
            
            # shuffle dataset
            idx = np.arange(n)
            np.random.shuffle(idx)
            x_shuffled = x[idx]
            y_shuffled = y[idx]

            for i in range(n):
                dp_x = x_shuffled[i]
                dp_y = y_shuffled[i]
                
                loss, W_1_grad, W_2_grad, b_1_grad, b_2_grad = self.loss(dp_x.reshape(1,-1), dp_y.reshape(1,-1))
                
                self.W_1 -= lr * W_1_grad
                self.W_2 -= lr * W_2_grad
                self.b_1 -= lr * b_1_grad
                self.b_2 -= lr * b_2_grad

                loss_history.append(loss)
            
            
            # print mean loss of dataset every 10% of epochs
            e = int(epochs / 10)
            if epoch % e == 0:
                print(f'Epoch {epoch}, Loss: {np.mean(loss_history[-n:])}')

        
        # TODO: Plot the loss history and return the loss achieved after the final epoch
        # Plot the loss history after every epoch. Returned is the mean loss of the last epoch.
        loss_per_epoch = [np.mean(loss_history[i*n:(i+1)*n]) for i in range(epochs)]
        plt.plot(loss_per_epoch)
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training Loss')
        plt.show()

        return loss_per_epoch[-1]

Epoch 0, Loss: 2.3048135152827944
Epoch 100, Loss: 0.004766474955605456
Epoch 200, Loss: 0.001929123905124359
Epoch 300, Loss: 0.0011586061105498446
Epoch 400, Loss: 0.0008842943399885283
Epoch 500, Loss: 0.0007160753823985639
Epoch 600, Loss: 0.0005888245134945728
Epoch 700, Loss: 0.000526121102732088
Epoch 800, Loss: 0.0005047205437658141
Epoch 900, Loss: 0.0004532285108616859


loss after final epoch: 0.00043452530197944737
Example predictions of model: 
x:0.4 y:[[0.1717219]]
x:0.5 y:[[0.24908155]]
x:0.7 y:[[0.49592807]]

 if these examples are predicted badly, the model might only perform well on training set


In [69]:
# We test the model created above on the simple function y = x^2

model = NumPyNeuralNet(1, 30, 1)

# Create a randomly distributed array of 1000 values between 0 and 1
x_train = 1 * np.random.randn(1000, 1)
# Create ground truth by calculating x*x
y_train = x_train * x_train

# Train for default epochs
loss = model.train(x_train, y_train)
print("loss after final epoch: " + str(loss))
example_1 = 0.4
example_2 = 0.5
example_3 = 0.7
print("Example predictions of model: " + "\n" + "x:" + str(example_1) + " y:" + str(model.predict(example_1)) + "\n"
      + "x:" + str(example_2) + " y:" + str(model.predict(example_2)) + "\n" + "x:" + str(example_3) + " y:" + str(model.predict(example_3)))
print("\n if these examples are predicted badly, the model might only perform well on training set")

## Embeddings

### Intrinsic evaluation of embeddings
Word similarity task is often used as an intrinsic evaluation criteria. In the dataset file you will find a list of word pairs with their similarity scores as judged by humans. The task would be to judge how well are the word vectors aligned to human judgement. We will use word2vec embedding vectors trained on the google news corpus. (Ignore the pairs where at least one the words is absent in the corpus)

In [1]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [2]:
# additional libraries from students
from gensim.models import Word2Vec
import torch as torch
import numpy as np
import csv
from torchmetrics import SpearmanCorrCoef

In [3]:
"""
Zelle zum testen/rumprobieren

# discard word pairs where at least one word is absent 
def test():
    scores = []
    # scores = torch.tensor([])

    cos = nn.CosineSimilarity(dim=0)
    prev_word = ""
    for index, word in enumerate(wv.index_to_key):
        # empty words
        if not prev_word:
            prev_word = word
            continue

        # word2vec as tensors
        # w1 = torch.from_numpy(np.array([wv[prev_word]]))
        # w2 = torch.from_numpy(np.array([wv[word]]))
        w1 = torch.from_numpy(wv[prev_word])
        w2 = torch.from_numpy(wv[word])
        

        # print(w1.shape)
        # print(w2.shape)
        cos_score = cos(w1, w2)
        # print(cosScore)
        # print(w1)
        # print(w2)
        scores.append([prev_word, word, cos_score])
        # scores = torch.cat((prev_word, word, cos_score.reshape(1)))
        # print(f"prev_word: {prev_word}, word: {word}, with score: {cos_score}")
        if index >= 10:
            break
        # print(scores)
        prev_word = word
    # print(scores)
    scores_sorted = sorted(scores, key=lambda a_entry: a_entry[2]) 
    # print(a)

    return scores_sorted
            

# scores = test()
# print(scores)
"""

'\nZelle zum testen/rumprobieren\n\n# discard word pairs where at least one word is absent \ndef test():\n    scores = []\n    # scores = torch.tensor([])\n\n    cos = nn.CosineSimilarity(dim=0)\n    prev_word = ""\n    for index, word in enumerate(wv.index_to_key):\n        # empty words\n        if not prev_word:\n            prev_word = word\n            continue\n\n        # word2vec as tensors\n        # w1 = torch.from_numpy(np.array([wv[prev_word]]))\n        # w2 = torch.from_numpy(np.array([wv[word]]))\n        w1 = torch.from_numpy(wv[prev_word])\n        w2 = torch.from_numpy(wv[word])\n        \n\n        # print(w1.shape)\n        # print(w2.shape)\n        cos_score = cos(w1, w2)\n        # print(cosScore)\n        # print(w1)\n        # print(w2)\n        scores.append([prev_word, word, cos_score])\n        # scores = torch.cat((prev_word, word, cos_score.reshape(1)))\n        # print(f"prev_word: {prev_word}, word: {word}, with score: {cos_score}")\n        if index >= 

In [4]:
def load_data():
    data = []      
    isFirstLine = True
    for line in open('wordsim353_dataset.csv'):
        if isFirstLine:
            isFirstLine = False
            continue
        indices = [i for i, x in enumerate(line) if x == ","]
        w1 = line[:indices[0]]
        w2 = line[indices[0]+1:indices[1]]
        mean = float(line[indices[1]+1:].rstrip())
        # print(f"w1: {w1}, w2: {w2}, mean: {mean}")
        data.append((w1, w2, mean))
    return data

# data = load_data()
# print(data)

Write a function which takes as input two words and computes the cosine similarity between them.
You do not need to implement the cosine similarity calculation from scratch. Feel free to use any Python library.
Remeber to ignore any pairs where at least one word is absent in the corpus. (2 points)

In [5]:
def similarity(word1, word2):
    cos = nn.CosineSimilarity(dim=0)
    output = cos(word1, word2)
    return output


Compute the similarity between all the word pairs in the list and sort them based on the similarity scores. (1 point)

In [6]:
def word2vec(w):
    return torch.from_numpy(wv[w])

In [7]:
def compute_similarity_scores(data):
    # data = load_data()
    scores = []
    
    # # scores = torch.tensor([])

    for w1, w2, mean in data:
        # empty words
        if not w1 or not w2:
            continue

        # word2vec as tensors for cos_score
        w1_vec = torch.from_numpy(wv[w1])
        w2_vec = torch.from_numpy(wv[w2])
        cos_score = similarity(w1_vec, w2_vec)
        
        scores.append([w1, w2, cos_score])
        # # scores = torch.cat((prev_word, word, cos_score.reshape(1)))
    return sorted(scores, key=lambda entry: entry[2])
# sim_scores = compute_similarity_scores(load_data())
# print(len(sim_scores))
# print(sim_scores)

Sort the word pairs in the list based on the human judgement scores. (1 point)

In [8]:
def human_judgement_scores(data):
    # data = load_data()
    return sorted(data, key=lambda entry: entry[2])
# human_scores = human_judgement_scores(loaddata())
# print(human_scores)

Compute spearman rank correlation between the two ranked lists obtained in the previous two steps.
You do not need to implement the spearman rank correlation calculation from scratch. Feel free to use any Python library. (2 points)

In [9]:
def compute_spearman():
    # data = load_data()
    # print(data)
    # spearman = SpearmanCorrCoef()
    # # spearman(preds, target)
    # spearman(var1, var2)
    pass

### Word embedding based clasifier
We will design a simple sentiment classifier based on the pre-trained word embeddings (google news).

Each data point is a movie review and the sentiment could be either positive (1) or negative (0)

In [10]:
import pickle
with open('sentiment_test_X.p', 'rb') as fs:
    test_X = pickle.load(fs)

len(test_X)

1821

In [ ]:
test_X[0]

In [ ]:
with open('sentiment_test_y.p', 'rb') as fs:
    test_y = pickle.load(fs)
    
len(test_y)

In [ ]:
test_y[0]

In [ ]:
with open('sentiment_train_X.p', 'rb') as fs:
    train_X = pickle.load(fs)
with open('sentiment_train_y.p', 'rb') as fs:
    train_y = pickle.load(fs)
with open('sentiment_val_X.p', 'rb') as fs:
    val_X = pickle.load(fs)
with open('sentiment_val_y.p', 'rb') as fs:
    val_y = pickle.load(fs)        

Given a review, compute its embedding by averaging over the embedding of its constituent words. Define a function which given a review as a list of words, generates its embeddings by averaging over the constituent word embeddings. (4 points)

In [ ]:
def generate_embedding(review):
    # return embedding
    pass

Create a feed-forward network class with pytorch. (Hyperparamter choice such as number of layers, hidden size is left to you) (8 points)

In [ ]:
class Classifier(nn.Module):
    pass

Create a Dataset class for efficiently enumerating over the dataset. (3 points)

In [ ]:
class sent_data(Dataset):
    def __init__(self):
        pass
    
    def __len__(self):
        pass
    
    def __getitem__(self):
        pass

Write a train function to train model. At the end of each epoch compute the validation accuracy and save the model with the best validation accuracy. (12 points)

In [ ]:
# Adopt your code to incorporate mini-batch training
# Use cross-entropy as your loss function
def train(model, train_data, val_data, epochs=5, learning_rate=0.001):
    # write your code snippet here
    
    pass

Evaluate the trained model on the test set and report the test accuracy. (3 points)

In [ ]:
def evaluate(model, test_data):
    pass